In [8]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from tqdm import tqdm
import os
import re
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import tensorflow as tf

dataset_path = 'DDS_Data_SEU'

PGB_path = '/Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/PGB'
RGB_path = '/Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/RGB'

# Specify the CSV file path
csv_file = '/Volumes/DataBoi/DDS_paper/data_robust.csv'

np.random.seed(45)

# Set the chunk size for reading the CSV
chunk_size = 100000  # Adjust the chunk size according to your memory limitations

In [3]:
def extract_fault(file_name):
    match = re.search(r'\d+', file_name)
    if match:
        return int(match.group(0)[0])  # Extract the first digit
    else:
        return None

def process_files_to_csv(data_folders, output_file):
    # Check if the file already exists
    if os.path.isfile(output_file):
        print(f"File {output_file} already exists. Skipping processing.")
        return

    total_files = sum([len(files) for data_folder in data_folders for r, d, files in os.walk(data_folder)])

    with open(output_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Channel1', 'Channel2', 'Channel3', 'Channel4', 'Channel5', 'Channel6', 'Channel7', 'Channel8', 'Speed', 'Type', 'Fault'])

        with tqdm(total=total_files, desc="Processing files", unit="file") as pbar:
            for data_folder in data_folders:
                for root, dirs, files in os.walk(data_folder):
                    if '.ipynb_checkpoints' in root:
                        continue  # Skip .ipynb_checkpoints folders
                    for file in files:
                        if file.endswith('.txt'):
                            file_path = os.path.join(root, file)
                            path_parts = file_path.split('/')
                            variable_speed = 'Variable_speed' in file_path
                            type_index = -4 if variable_speed else -3
                            type = path_parts[type_index] if path_parts[type_index] in ['PGB', 'RGB'] else None
                            if type is not None:
                                speed_index = -3 if variable_speed else -2
                                speed = path_parts[speed_index]
                                fault = extract_fault(file)

                                data = pd.read_csv(file_path, sep='\t', encoding='ISO-8859-1')
                                reshaped_data = data.values[:, :]

                                for row_data in tqdm(reshaped_data, desc="Processing rows", unit="row", leave=False):
                                    row = row_data.tolist() + [speed, type, fault]
                                    csv_writer.writerow(row)
                            pbar.update()

In [4]:
# Example usage
process_files_to_csv([PGB_path, RGB_path], '/Volumes/DataBoi/DDS_paper/data_robust.csv')

File /Volumes/DataBoi/DDS_paper/data_robust.csv already exists. Skipping processing.


# Data preprocessing

In [ ]:
import pandas as pd
from tqdm import tqdm

# Initialize a dictionary to store the fault counts
fault_counts = {}

# Iterate through the CSV file using chunksize
with tqdm(total=1, unit='chunk', desc='Processing CSV') as pbar:
    for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
        
        #print(chunk)
        # Assuming there is a column named 'fault' in the CSV representing the fault type
        fault_column = 'Fault'

        # Count the occurrences of each fault in the current chunk
        fault_chunk_counts = chunk[fault_column].value_counts()

        # Aggregate the counts with the overall fault_counts dictionary
        for fault, count in fault_chunk_counts.items():
            fault_counts[fault] = fault_counts.get(fault, 0) + count

        pbar.update()

# Print the fault counts
for fault, count in fault_counts.items():
    print(f"Fault: {fault}, Count: {count}")


In [ ]:
import pandas as pd
from tqdm import tqdm

# Initialize dictionaries to store the counts
speed_counts = {}
type_counts = {}

# Iterate through the CSV file using chunksize
with tqdm(total=1, unit='chunk', desc='Processing CSV') as pbar:
    for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
        # Assuming there is a column named 'Speed' in the CSV representing the speed values
        
        speed_column = 'Speed'

        # Count the occurrences of each speed value in the current chunk
        speed_chunk_counts = chunk[speed_column].value_counts()

        # Aggregate the counts with the overall speed_counts dictionary
        for speed, count in speed_chunk_counts.items():
            speed_counts[speed] = speed_counts.get(speed, 0) + count

        # Assuming there is a column named 'Type' in the CSV representing the types
        type_column = 'Type'

        # Count the occurrences of each type in the current chunk
        type_chunk_counts = chunk[type_column].value_counts()

        # Aggregate the counts with the overall type_counts dictionary
        for typ, count in type_chunk_counts.items():
            type_counts[typ] = type_counts.get(typ, 0) + count

        pbar.update()

# Print the speed counts
print("Speed Counts:")
for speed, count in speed_counts.items():
    print(f"Speed: {speed}, Count: {count}")

# Print the type counts
print("Type Counts:")
for typ, count in type_counts.items():
    print(f"Type: {typ}, Count: {count}")


In [5]:
# specify the numeric and categorical features
numerical_features = ['Channel1', 'Channel2', 'Channel3', 'Channel4', 'Channel5', 'Channel6', 'Channel7', 'Channel8']
categorical_features = ['Speed', 'Type']

# initialize the scalers
scaler = StandardScaler()
encoder = OneHotEncoder(categories=[['20_0','30_0','30_1','30_2', '30_3','30_4','30_5','40_0','50_0', 'Variable_speed'], ['PGB', 'RGB']])  # specify the unique values

In [6]:
# define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numerical_features),
        ('cat', encoder, categorical_features)])


In [9]:
# get the total size of the file in bytes
total_size = os.path.getsize(csv_file)

# define the number of lines to read for the estimate
sample_lines = 1000

# read the first few lines of the file to get the average bytes per line
with open(csv_file, 'r') as file:
    lines = [next(file) for _ in range(sample_lines)]
avg_bytes_per_line = sum(len(line) for line in lines) / sample_lines

# estimate the total number of lines in the file
estimated_lines = total_size // avg_bytes_per_line

# calculate the estimated total number of chunks
total_chunks = estimated_lines // chunk_size
if estimated_lines % chunk_size != 0:
    total_chunks += 1
    
# generate a boolean mask for the entire dataset
mask = np.random.rand(int(estimated_lines)) < 0.8


In [5]:
# initialize a progress bar
progress_bar = tqdm(total=total_chunks, desc="Fitting Scaler and Encoder", unit="chunk")

# fit the scaler and encoder incrementally
for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
    chunk_mask = np.random.rand(len(chunk)) < 0.8
    train_chunk = chunk[chunk_mask]
    scaler.partial_fit(train_chunk[numerical_features])
    encoder.fit(train_chunk[categorical_features])
    progress_bar.update()

# close the progress bar
progress_bar.close()

Fitting Scaler and Encoder:  98%|█████████▊| 72667/73892.0 [12:19<00:12, 98.21chunk/s] 


In [6]:
progress_bar = tqdm(total=total_chunks, desc="Fitting preprocessor", unit="chunk")

# fit the preprocessor on the entire training data
for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
    chunk_mask = np.random.rand(len(chunk)) < 0.8
    train_chunk = chunk[chunk_mask]
    preprocessor.fit(train_chunk)
    progress_bar.update()
# close the progress bar
progress_bar.close()

Fitting preprocessor:  98%|█████████▊| 72667/73892.0 [14:26<00:14, 83.91chunk/s] 


In [21]:
# define a generator function for the training set
def train_generator():
    for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
        chunk_mask = np.random.rand(len(chunk)) < 0.8
        train_chunk = chunk[chunk_mask]
        if len(train_chunk) == chunk_size:  # only yield full chunks
            # perform the transformations
            transformed_chunk = preprocessor.transform(train_chunk)
            yield transformed_chunk

# define a generator function for the validation set
def val_generator():
    for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
        chunk_mask = np.random.rand(len(chunk)) < 0.8
        val_chunk = chunk[~chunk_mask]
        if len(val_chunk) == chunk_size:  # only yield full chunks
            # perform the transformations
            transformed_chunk = preprocessor.transform(val_chunk)
            yield transformed_chunk 


In [22]:
# create tf.data.Datasets for training and validation
train_dataset = tf.data.Dataset.from_generator(train_generator, output_signature=(
    tf.TensorSpec(shape=(None, 20), dtype=tf.float32)))

val_dataset = tf.data.Dataset.from_generator(val_generator, output_signature=(
    tf.TensorSpec(shape=(None, 20), dtype=tf.float32)))


In [1]:
# train the model on the training dataset
for train_batch in train_dataset.batch(2):
    # perform a training step here
    print(train_batch)
    break

NameError: name 'train_dataset' is not defined